In [2]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.5.5


In [3]:
input_file = "input.txt"

# Part 1

In [4]:
gds.run_cypher("""CREATE INDEX point_row_col IF NOT EXISTS
FOR (p:Point) ON (p.row, p.col)""")

gds.run_cypher("""CREATE INDEX port_row_col_speed_ori IF NOT EXISTS
FOR (p:Point) ON (p.row, p.col, p.speed, p.orientation)""")

""


In [6]:
lines = [{'ix':i, 'line':l[:-1]} for i, l in enumerate(open(input_file, 'r').readlines(), 1)]

gds.run_cypher("""
UNWIND $lines AS line
WITH line.ix AS ix, line.line AS line
CALL {
WITH ix, line
WITH ix, split(line,'') AS line
WITH [col IN range(0,size(line)-1) | {row:ix, col:col+1, heat_loss:toInteger(line[col])}] AS line
UNWIND line AS point
CREATE (p:Point:Block) SET p = point
} IN TRANSACTIONS OF 10000 ROWS
""", {'lines':lines})

""


In [7]:
gds.run_cypher("""
MATCH (n:Block)
CALL {
WITH n
UNWIND ['N','S','E','W'] AS dir
UNWIND ['+','-'] AS orientation
UNWIND range(0,3) AS speed
MERGE (p:Point:Port {row:{N:-0.5, S:0.5, E:0, W:0}[dir] + n.row,
    col:{W:-0.5, E:0.5, S:0, N:0}[dir] + n.col,
    speed:speed,
    orientation: orientation})
MERGE (n)-[:HAS_PORT {dir:dir}]->(p)
} IN TRANSACTIONS OF 1000 ROWS
""")

""


In [8]:
gds.run_cypher("""
MATCH (p1)-[r1:HAS_PORT WHERE r1.dir in ['N','W']]–(b:Block)
    -[r2 WHERE r2.dir = {N:'S',W:'E'}[r1.dir]]->(p2)
WHERE p1.speed IN [0,1,2] AND p1.orientation = '+'
AND p2.speed = p1.speed + 1 AND p2.orientation = '+'
CALL {
WITH p1, p2, b
MERGE (p1)-[:STEP {heat_loss:b.heat_loss}]->(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")

""


In [9]:
            
gds.run_cypher("""
MATCH (p1)-[r1:HAS_PORT WHERE r1.dir in ['N','W']]–(b:Block)
    -[r2 WHERE r2.dir = {N:'S',W:'E'}[r1.dir]]->(p2)
WHERE p2.speed IN [0,1,2] AND p2.orientation = '-'
AND p1.speed = p2.speed + 1 AND p1.orientation = '-'
CALL {
WITH p1, p2, b
MERGE (p1)<-[:STEP {heat_loss:b.heat_loss}]-(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")


""


In [10]:
               
gds.run_cypher("""
MATCH (p1 WHERE p1.orientation = '+')<-[r1:HAS_PORT WHERE r1.dir in ['N','W']]–(b:Block)
    -[r2:HAS_PORT WHERE r2.dir IN {N:['E','W'],W:['N','S']}[r1.dir]]->(p2)
WHERE p2.speed = 1 AND p2.orientation = CASE r2.dir IN ['N','W'] WHEN TRUE THEN '-' ELSE '+' END 
CALL {
WITH p1, p2, b
MERGE (p1)-[:STEP {heat_loss:b.heat_loss}]->(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")

""


In [11]:

gds.run_cypher("""
MATCH (p1 WHERE p1.orientation = '-')<-[r1:HAS_PORT WHERE r1.dir in ['S','E']]–(b:Block)
    -[r2:HAS_PORT WHERE r2.dir IN {S:['E','W'],E:['N','S']}[r1.dir]]->(p2)
WHERE p2.speed = 1 AND p2.orientation = CASE r2.dir IN ['N','W'] WHEN TRUE THEN '-' ELSE '+' END 
CALL {
WITH p1, p2, b
MERGE (p1)-[:STEP {heat_loss:b.heat_loss}]->(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")

""


In [12]:
gds.run_cypher("""
MERGE (start:Point:Start {row:0, col:0})
WITH start
MATCH (b:Block {row:1, col:1})-[r:HAS_PORT WHERE r.dir IN ['S', 'E']]->(p:Port {speed:1, orientation:'+'})
MERGE (start)-[:STEP {heat_loss:0}]->(p);
""")

gds.run_cypher("""
MATCH (any_point:Point:Block)
WITH max(any_point.col) AS mxc, max(any_point.row) AS mxr
MERGE (last:Point:Last {col: mxc+0.5, row:mxr+0.5})
WITH last, mxc, mxr
MATCH (bottom_right:Point:Block {col: mxc, row:mxr})-[r:HAS_PORT WHERE r.dir IN ['S', 'E']]->(p:Port)
MERGE (p)-[:STEP {heat_loss:0}]->(last);
""")

""


In [13]:
gds.run_cypher("""
MATCH (source:Start|Last|Port)
OPTIONAL MATCH (source)-[r:STEP]->(target)
WITH gds.graph.project(
  'map',
  source,
  target,
  { relationshipProperties: r { .heat_loss } }
) AS g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
""")

,graph,nodes,rels
0,map,320354,874782


In [15]:
gds.run_cypher("""
MATCH (source:Start), (target:Last)
CALL gds.shortestPath.dijkstra.stream('map', {
    sourceNode: source,
    targetNode: target,
    relationshipWeightProperty: 'heat_loss'
})
YIELD totalCost
RETURN toInteger(totalCost) AS part1
""")

,part1
0,1004


# Part 2

In [5]:
lines = [{'ix':i, 'line':l[:-1]} for i, l in enumerate(open(input_file, 'r').readlines(), 1)]

gds.run_cypher("""
UNWIND $lines AS line
WITH line.ix AS ix, line.line AS line
CALL {
WITH ix, line
WITH ix, split(line,'') AS line
WITH [col IN range(0,size(line)-1) | {row:ix, col:col+1, heat_loss:toInteger(line[col])}] AS line
UNWIND line AS point
CREATE (p:Point:Block) SET p = point
} IN TRANSACTIONS OF 10000 ROWS
""", {'lines':lines})

""


In [6]:
gds.run_cypher("""
MATCH (n:Block)
CALL {
WITH n
UNWIND ['N','S','E','W'] AS dir
UNWIND ['+','-'] AS orientation
UNWIND range(0,10) AS speed
MERGE (p:Point:Port {row:{N:-0.5, S:0.5, E:0, W:0}[dir] + n.row,
    col:{W:-0.5, E:0.5, S:0, N:0}[dir] + n.col,
    speed:speed,
    orientation: orientation})
MERGE (n)-[:HAS_PORT {dir:dir}]->(p)
} IN TRANSACTIONS OF 1000 ROWS
""")

""


In [7]:
gds.run_cypher("""
MATCH (p1)-[r1:HAS_PORT WHERE r1.dir in ['N','W']]–(b:Block)
    -[r2 WHERE r2.dir = {N:'S',W:'E'}[r1.dir]]->(p2)
WHERE p1.speed IN range(0,9) AND p1.orientation = '+'
AND p2.speed = p1.speed + 1 AND p2.orientation = '+'
CALL {
WITH p1, p2, b
MERGE (p1)-[:STEP {heat_loss:b.heat_loss}]->(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")

""


In [8]:
            
gds.run_cypher("""
MATCH (p1)-[r1:HAS_PORT WHERE r1.dir in ['N','W']]–(b:Block)
    -[r2 WHERE r2.dir = {N:'S',W:'E'}[r1.dir]]->(p2)
WHERE p2.speed IN range(0,9) AND p2.orientation = '-'
AND p1.speed = p2.speed + 1 AND p1.orientation = '-'
CALL {
WITH p1, p2, b
MERGE (p1)<-[:STEP {heat_loss:b.heat_loss}]-(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")


""


In [9]:
gds.run_cypher("""
MATCH (p1 WHERE p1.orientation = '+' AND p1.speed >= 4)<-[r1:HAS_PORT WHERE r1.dir in ['N','W']]–(b:Block)
    -[r2:HAS_PORT WHERE r2.dir IN {N:['E','W'],W:['N','S']}[r1.dir]]->(p2)
WHERE p2.speed = 1 AND p2.orientation = CASE r2.dir IN ['N','W'] WHEN TRUE THEN '-' ELSE '+' END 
CALL {
WITH p1, p2, b
MERGE (p1)-[:STEP {heat_loss:b.heat_loss}]->(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")

""


In [10]:
gds.run_cypher("""
MATCH (p1 WHERE p1.orientation = '-' AND p1.speed >= 4)<-[r1:HAS_PORT WHERE r1.dir in ['S','E']]–(b:Block)
    -[r2:HAS_PORT WHERE r2.dir IN {S:['E','W'],E:['N','S']}[r1.dir]]->(p2)
WHERE p2.speed = 1 AND p2.orientation = CASE r2.dir IN ['N','W'] WHEN TRUE THEN '-' ELSE '+' END 
CALL {
WITH p1, p2, b
MERGE (p1)-[:STEP {heat_loss:b.heat_loss}]->(p2)
} IN TRANSACTIONS OF 1000 ROWS;
""")

""


In [11]:
gds.run_cypher("""
MERGE (start:Point:Start {row:0, col:0})
WITH start
MATCH (b:Block {row:1, col:1})-[r:HAS_PORT WHERE r.dir IN ['S', 'E']]->(p:Port {speed:1, orientation:'+'})
MERGE (start)-[:STEP {heat_loss:0}]->(p);
""")

gds.run_cypher("""
MATCH (any_point:Point:Block)
WITH max(any_point.col) AS mxc, max(any_point.row) AS mxr
MERGE (last:Point:Last {col: mxc+0.5, row:mxr+0.5})
WITH last, mxc, mxr
MATCH (bottom_right:Point:Block {col: mxc, row:mxr})-[r:HAS_PORT WHERE r.dir IN ['S', 'E']]->(p:Port WHERE p.speed > 4 )
MERGE (p)-[:STEP {heat_loss:0}]->(last);
""")

""


In [12]:
gds.run_cypher("""
MATCH (source:Start|Last|Port)
OPTIONAL MATCH (source)-[r:STEP]->(target)
WITH gds.graph.project(
  'map',
  source,
  target,
  { relationshipProperties: r { .heat_loss } }
) AS g
RETURN g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
""")

,graph,nodes,rels
0,map,880970,1908602


In [13]:
gds.run_cypher("""
MATCH (source:Start), (target:Last)
CALL gds.shortestPath.dijkstra.stream('map', {
    sourceNode: source,
    targetNode: target,
    relationshipWeightProperty: 'heat_loss'
})
YIELD totalCost
RETURN toInteger(totalCost) AS part2
""")

,part2
0,1171
